# Mini Dollar Strategy - Backtesting and Optimization

This notebook demonstrates:
1. Strategy backtesting
2. Performance analysis
3. Parameter optimization
4. Results visualization

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from itertools import product
from tqdm import tqdm

from src.agents.coordinator import StrategyCoordinator
from src.reporting.performance import PerformanceAnalyzer

# Configure plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 6]
sns.set_palette('Set2')

## 1. Initial Strategy Test

In [ ]:
# Initialize components
coordinator = StrategyCoordinator(db_path="../src/data/database/candles.db")
analyzer = PerformanceAnalyzer()

# Run backtest
end_date = datetime.now()
start_date = end_date - timedelta(days=180)

results = coordinator.backtest(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'),
    interval=5  # 5 minutos
)

# Get performance metrics
metrics = analyzer.calculate_metrics(results)
print("\nPerformance Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")

## 2. Detailed Performance Analysis

In [ ]:
# Generate detailed performance report
report = analyzer.generate_report(results)
print(report)

# Plot equity curve and drawdowns
analyzer.plot_equity_curve(results)

def plot_trade_distribution(results):
    """Plot distribution of trade profits/losses."""
    trades = results[results['trade_executed']]['profit']
    
    plt.figure(figsize=(15, 5))
    sns.histplot(trades, bins=50)
    plt.title('Trade Profit/Loss Distribution')
    plt.xlabel('Points')
    plt.grid(True)
    plt.show()
    
    print("\nTrade Statistics:")
    print(f"Average Trade: {trades.mean():.2f}")
    print(f"Std Dev: {trades.std():.2f}")
    print(f"Best Trade: {trades.max():.2f}")
    print(f"Worst Trade: {trades.min():.2f}")

plot_trade_distribution(results)

## 3. Parameter Optimization

In [ ]:
def optimize_parameters(start_date: str, end_date: str) -> pd.DataFrame:
    """Grid search para otimização de parâmetros completos."""
    
    # Definição dos ranges de parâmetros
    param_ranges = {
        # RSI
        'rsi_periods': [9, 14, 21],
        'rsi_upper': [65, 70, 75],
        'rsi_lower': [25, 30, 35],
        
        # Médias Móveis
        'ma_fast_periods': [5, 8, 13],
        'ma_slow_periods': [21, 34, 55],
        'ma_types': ['EMA', 'SMA'],
        
        # Stops
        'stop_losses': [1.5, 2.0, 2.5],  # Multiplicadores do ATR
        'take_profits': [2.0, 2.5, 3.0],  # Multiplicadores do ATR
        'trailing_stops': [1.0, 1.5, 2.0],  # Multiplicadores do ATR
        
        # ADX
        'adx_thresholds': [20, 25, 30],
        
        # Score
        'min_scores': [3.0, 4.0, 5.0]
    }
    
    # Criar combinações de parâmetros principais
    param_combinations = list(product(
        param_ranges['rsi_periods'],
        param_ranges['ma_fast_periods'],
        param_ranges['ma_slow_periods'],
        param_ranges['stop_losses'],
        param_ranges['take_profits'],
        param_ranges['ma_types'],
        param_ranges['adx_thresholds'],
        param_ranges['min_scores']
    ))
    
    results = []
    total_iterations = len(param_combinations)
    start_time = time.time()
    analyzer = PerformanceAnalyzer()
    db_path = "../src/data/database/candles.db"

    print(f"\nIniciando otimização com {total_iterations} combinações")
    print(f"Período: {start_date} até {end_date}")
    
    for idx, params in enumerate(tqdm(param_combinations, desc="Otimizando Parâmetros")):
        try:
            (rsi_period, ma_fast, ma_slow, sl_mult, tp_mult, 
             ma_type, adx_thresh, min_score) = params
            
            # Validação de médias móveis
            if ma_fast >= ma_slow:
                continue
            
            # Configurar estratégia
            coord = StrategyCoordinator(
                initial_balance=100000,
                max_position=1,
                db_path=db_path,
                strategy_params={
                    'rsi_period': rsi_period,
                    'ma_fast': ma_fast,
                    'ma_slow': ma_slow,
                    'ma_type': ma_type,
                    'atr_multiplier': sl_mult,
                    'take_profit_multiplier': tp_mult,
                    'trailing_stop_multiplier': sl_mult,
                    'adx_threshold': adx_thresh,
                    'min_score_entry': min_score,
                    'use_trailing_stop': True
                }
            )
            
            # Executar backtest
            backtest_results = coord.backtest(
                start_date=start_date,
                end_date=end_date,
                interval=5
            )
            
            # Verificar resultados
            if backtest_results is None or backtest_results.empty:
                continue
            
            # Calcular métricas
            metrics = analyzer.calculate_metrics(backtest_results)
            
            # Registrar resultados
            results.append({
                # Parâmetros RSI
                'rsi_period': rsi_period,
                'rsi_upper': param_ranges['rsi_upper'][1],  # valor padrão
                'rsi_lower': param_ranges['rsi_lower'][1],  # valor padrão
                
                # Parâmetros MA
                'ma_fast': ma_fast,
                'ma_slow': ma_slow,
                'ma_type': ma_type,
                
                # Parâmetros Stops
                'stop_loss_mult': sl_mult,
                'take_profit_mult': tp_mult,
                'trailing_stop': sl_mult,
                
                # Outros parâmetros
                'adx_threshold': adx_thresh,
                'min_score': min_score,
                
                # Métricas
                'total_profit': metrics.get('total_profit', 0),
                'win_rate': metrics.get('win_rate', 0),
                'sharpe_ratio': metrics.get('sharpe_ratio', 0),
                'max_drawdown': metrics.get('max_drawdown', 0),
                'total_trades': metrics.get('total_trades', 0),
                'avg_trade': metrics.get('avg_trade', 0),
                'profit_factor': metrics.get('profit_factor', 0),
                'avg_win_loss_ratio': metrics.get('avg_win_loss_ratio', 0)
            })
            
            # Mostrar progresso a cada 50 iterações
            if (idx + 1) % 50 == 0:
                elapsed_time = time.time() - start_time
                iterations_left = total_iterations - (idx + 1)
                estimated_time_left = (elapsed_time / (idx + 1)) * iterations_left
                
                print(f"\nProgresso: {((idx + 1)/total_iterations)*100:.1f}%")
                print(f"Tempo estimado restante: {estimated_time_left/60:.1f} minutos")
                
        except Exception as e:
            print(f"\nErro na iteração {idx + 1}: {str(e)}")
            continue
    
    if not results:
        raise ValueError("Nenhuma iteração completada com sucesso")
    
    # Criar DataFrame com resultados
    results_df = pd.DataFrame(results)
    
    # Filtrar resultados
    results_df = results_df[
        (results_df['total_trades'] >= 10) &          # Mínimo de trades
        (results_df['win_rate'] >= 0.4) &             # Win rate mínimo de 40%
        (results_df['max_drawdown'].abs() < 0.25) &   # Drawdown máximo de 25%
        (results_df['profit_factor'] > 1.2)           # Profit factor mínimo de 1.2
    ]
    
    return results_df

# Executar otimização
try:
    # Usar período de 30 dias para teste inicial
    end_date = datetime(2024, 11, 29)  # Última data disponível
    start_date = end_date - timedelta(days=30)
    
    print(f"Iniciando otimização de {start_date.strftime('%Y-%m-%d')} até {end_date.strftime('%Y-%m-%d')}")
    
    # Carregar e verificar dados antes da otimização
    data_loader = MarketDataLoader("../src/data/database/candles.db")
    initial_data = data_loader.get_minute_data(
        interval=5,
        start_date=start_date.strftime('%Y-%m-%d'),
        end_date=end_date.strftime('%Y-%m-%d')
    )
    
    if initial_data.empty:
        raise ValueError("Nenhum dado disponível para o período selecionado")
        
    print(f"\nDados carregados:")
    print(f"Total de candles: {len(initial_data)}")
    print(f"Período: {initial_data.index.min()} até {initial_data.index.max()}")
    
    # Executar otimização
    results = optimize_parameters(
        start_date=start_date.strftime('%Y-%m-%d'),
        end_date=end_date.strftime('%Y-%m-%d')
    )
    
    if not results.empty:
        print("\nMelhores combinações por Sharpe Ratio:")
        display_columns = [
            'total_profit', 'win_rate', 'sharpe_ratio', 'profit_factor',
            'max_drawdown', 'total_trades', 'avg_trade', 'ma_type',
            'rsi_period', 'ma_fast', 'ma_slow', 'stop_loss_mult',
            'take_profit_mult', 'min_score'
        ]
        
        print("\nTop 10 por Sharpe Ratio:")
        print(results.sort_values('sharpe_ratio', ascending=False)[display_columns].head(10))
        
        print("\nTop 10 por Profit Factor:")
        print(results.sort_values('profit_factor', ascending=False)[display_columns].head(10))
        
        print("\nTop 10 por Total Profit (com Sharpe > 1):")
        profit_results = results[results['sharpe_ratio'] > 1].sort_values('total_profit', ascending=False)
        print(profit_results[display_columns].head(10))
        
        # Salvar resultados
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        results.to_csv(f"optimization_results_{timestamp}.csv", index=False)
        print(f"\nResultados salvos em: optimization_results_{timestamp}.csv")
        
    else:
        print("Nenhum resultado encontrado que atenda aos critérios mínimos")
    
except Exception as e:
    print(f"Erro durante o processo de otimização: {str(e)}")
    import traceback
    traceback.print_exc()

## 4. Analysis of Optimal Parameters

In [ ]:
def analyze_parameter_impact(results_df):
    """Analyze the impact of different parameters on performance."""
    # RSI Period Analysis
    plt.figure(figsize=(15, 5))
    sns.boxplot(x='rsi_period', y='total_profit', data=results_df)
    plt.title('RSI Period vs Total Profit')
    plt.show()
    
    # Moving Averages Analysis
    plt.figure(figsize=(15, 5))
    sns.scatterplot(x='ma_fast', y='ma_slow', size='total_profit', 
                    hue='win_rate', data=results_df)
    plt.title('Moving Averages Periods vs Performance')
    plt.show()
    
    # Stop Loss and Take Profit Analysis
    plt.figure(figsize=(15, 5))
    sns.scatterplot(x='stop_loss', y='take_profit', size='total_profit',
                    hue='sharpe_ratio', data=results_df)
    plt.title('Stop Loss vs Take Profit')
    plt.show()

# Analyze parameter impact
analyze_parameter_impact(optimization_results)

# Get best parameters
best_params = optimization_results.loc[optimization_results['total_profit'].idxmax()]
print("\nBest Parameters:")
print(f"RSI Period: {best_params['rsi_period']}")
print(f"Fast MA: {best_params['ma_fast']}")
print(f"Slow MA: {best_params['ma_slow']}")
print(f"Stop Loss: {best_params['stop_loss']}")
print(f"Take Profit: {best_params['take_profit']}")

## 5. Out-of-Sample Testing

In [ ]:
db_path = "../src/data/database/candles.db" 

# Test optimized strategy on recent data
coordinator_optimized = StrategyCoordinator(
    initial_balance=100000,
    max_position=1,
    stop_loss=best_params['stop_loss'],
    take_profit=best_params['take_profit'],
    db_path=db_path
)

# Run backtest on recent data
end_date = datetime.now()
start_date = end_date - timedelta(days=30)  # Last month

recent_results = coordinator_optimized.backtest(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'),
    interval=5
)

# Calculate and show performance
recent_metrics = analyzer.calculate_metrics(recent_results)
print("\nOut-of-Sample Performance:")
for metric, value in recent_metrics.items():
    print(f"{metric}: {value:.2f}")

# Plot equity curve
analyzer.plot_equity_curve(recent_results)